In [42]:
import pandas as pd
import numpy as np
import math
from copy import deepcopy


#############################
# Import and combine all data
#############################


#DATA_PATH = "/Users/mvonebers/HUBBS-Lab/data/"
DATA_PATH = "/home/maggie/HUBBS-Lab/data/"

e4_data = pd.read_excel(DATA_PATH + "E4_TEST.xlsx")
change_data = pd.read_excel(DATA_PATH + "normalized_change.xlsx")
audio_data = pd.read_excel(DATA_PATH + "audio_TEST.xlsx")
demo_data = pd.read_csv(DATA_PATH + "Demographics Information.csv")

# Break apart the ID column into "person" and "trial"
def clean_id(data):
    data.insert(0, "person", [0] * data.shape[0])
    data.insert(1, "trial", [0] * data.shape[0])

    for i, row in data.iterrows():
        data.at[i, "person"] = int(data.at[i, "id"][7:])
        data.at[i, "trial"] = int(data.at[i, "id"][5])
    
    data = data.drop(columns=['id'])
    data = data.rename(columns={"person": "id"})
    return data

    
e4_data = clean_id(e4_data)
audio_data = clean_id(audio_data)    

# for col in audio_data.columns.to_list():
#     if col != 'id' and col != 'trial':
#         audio_data[col] = audio_data[col]/audio_data[col].max()
        
# for col in e4_data.columns.to_list():
#     if col != 'id' and col != 'trial':
#         e4_data[col] = e4_data[col]/e4_data[col].max()
    
all_data = pd.merge(e4_data, change_data, on='id')
all_data = audio_data.merge(all_data, how='right')


# Reorder survey data in order of most samples to least
columns = all_data.columns.to_list()
new_columns = deepcopy(columns)
new_columns[35] = columns[40]
new_columns[37] = columns[41] 
new_columns[38] = columns[37] 
new_columns[40] = columns[35]
new_columns[41] = columns[38]

all_data = all_data[new_columns]


# what does this do?
#demo_ids = demo_data['id'].to_list()
#
#for id_ in demo_ids:
#    if id_ not in slope_ids:
#        demo_data = demo_data[demo_data.id != id_]
        
all_data = all_data.merge(demo_data, how="right")

# The demographic data lists IDs that aren't present in the other data, so remove them
all_data = all_data[all_data.id != 16]
all_data = all_data[all_data.id != 27]
all_data = all_data[all_data.id != 38]
all_data = all_data[all_data.id != 43]
all_data = all_data[all_data.id != 46]
all_data = all_data[all_data.id != 49]
all_data = all_data[all_data.id != 53]
all_data = all_data[all_data.id != 58]
all_data = all_data[all_data.id != 65]
all_data = all_data[all_data.id != 66]

all_data.drop(['CAI Trait Small group Score', 'CAI Trait Dyadic Score', 'CAI Trait Public Speaking Score', 'STAI State Score', 'Brief fear of Negative Evaluation'], axis=1, inplace=True)

In [43]:
###############################################################
# Get slopes from linear regression of the 8 trials for each ID
###############################################################

from sklearn.linear_model import LinearRegression

def get_slopes(data, start, end):
    y0 = data['trial'].to_numpy(copy=True)

    slopes = pd.DataFrame(np.zeros((19, 45)), columns=data.columns)
    slopes = slopes.drop(["trial"], axis=1)

    for col in range(2, 34):
        x1 = data[data.columns[col]]
        y0 = list(range(start, end + 1))
        for row in range(19):
            x0 = x1[ (row * 8) + start - 1 : (row * 8) + end ].to_numpy()
            x = np.array([])
            y = np.array([])
            
            slopes.iloc[row, 0] = data.iloc[row * 8, 0]

            for i in range(len(x0)):  # remove NaN from data
                if not math.isnan(x0[i]) and not math.isnan(y0[i]):
                    x = np.append(x, x0[i])
                    y = np.append(y, y0[i])
            
            try:
                reg = LinearRegression().fit(y.reshape(-1,1),x)
                slopes.iloc[row, col - 1] = reg.coef_
            except:
                 slopes.iloc[row, col - 1] = 0
    
    for col in range(34, 45):
        for row in range(19):
            slopes.iloc[row, col - 1] = data.iloc[row * 8, col]
            
    # Want to preserve zeros in the demographic data, so temporarily boost it up one...
    for col in range(43, 45):
        for row in range(19):
            slopes.iloc[row, col - 1] += 1.0
                    
    slopes.replace(0, np.NaN, inplace=True)
    
    # Then bump it back down. (I know this is a dumb way to do this.)
    for col in range(43, 45):
        for row in range(19):
            slopes.iloc[row, col - 1] -= 1.0
    
    return slopes

all_slopes = get_slopes(all_data, 1, 8)

all_slopes

,id,pcm_RMSenergy_sma_amean,pcm_fftMag_mfcc_sma[1]_amean,pcm_fftMag_mfcc_sma[2]_amean,pcm_fftMag_mfcc_sma[3]_amean,pcm_fftMag_mfcc_sma[4]_amean,pcm_fftMag_mfcc_sma[5]_amean,pcm_fftMag_mfcc_sma[6]_amean,pcm_fftMag_mfcc_sma[7]_amean,pcm_fftMag_mfcc_sma[8]_amean,...,CAI Trait Full Score,STAI Trait Score,Age,Gender,Lang,college,presentation,ethnicity,presentation_3_months,highest_education
0,4.0,-0.000864,-1.188091,1.458396,-0.644212,-0.697069,-0.543809,2.068849,-1.118681,0.597193,...,0.101695,-0.068182,3.0,1.0,2.0,1.0,2.0,1.0,0.0,3.0
1,5.0,-0.004605,0.022244,0.041306,0.371020,1.856301,-2.347585,2.857080,-0.310527,-0.021322,...,0.121212,0.025641,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0
2,8.0,0.001767,-0.545172,-0.233862,0.693634,-1.303848,-0.635333,0.913341,-0.946743,-0.024012,...,0.128205,0.155172,1.0,2.0,1.0,1.0,3.0,2.0,2.0,1.0
3,20.0,-0.003113,0.120467,-0.326470,0.884765,-0.090467,0.182058,-0.183817,0.728905,-0.347066,...,0.043478,0.040000,1.0,1.0,2.0,3.0,1.0,2.0,2.0,1.0
4,21.0,-0.000428,-0.201158,-0.268297,0.201903,-0.366338,0.377646,0.226589,-0.530547,0.462883,...,0.121212,0.114286,1.0,1.0,1.0,1.0,2.0,2.0,0.0,1.0
5,23.0,-0.002165,-0.024271,0.128524,0.470709,1.293796,-1.696811,3.038422,-0.332586,-0.556816,...,0.066667,0.100000,1.0,1.0,1.0,1.0,2.0,2.0,0.0,1.0
6,32.0,-0.001112,0.432318,-0.105907,1.495445,-1.006755,1.001877,0.911382,-0.181762,-0.036236,...,0.272727,-0.214286,2.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0
7,35.0,-0.003161,0.366671,0.193564,-0.023716,0.258827,0.975375,-0.922304,0.307595,-0.030484,...,NaN,-0.068966,1.0,2.0,1.0,3.0,4.0,1.0,2.0,1.0
8,37.0,-0.000485,0.394130,0.261597,-0.139036,0.246072,0.183514,0.162386,-0.175606,-0.127135,...,0.025641,-0.107143,1.0,1.0,NaN,4.0,2.0,3.0,1.0,1.0
9,41.0,-0.000964,-0.020902,0.018043,0.745624,0.887429,-3.439343,1.431305,-0.421041,-1.802264,...,0.138462,NaN,2.0,2.0,1.0,1.0,3.0,3.0,2.0,2.0


In [4]:
# Normalization
#for col_name in all_slopes.columns.to_list():
#    all_slopes[col_name] = all_slopes[col_name]/all_slopes[col_name].max()

#all_slopes["pause_frequency"] = all_slopes["pause_frequency"]/all_slopes["pause_frequency"].max()
#all_slopes["pause_interval"] = all_slopes["pause_interval"]/all_slopes["pause_interval"].max()
#all_slopes

,id,pcm_RMSenergy_sma_amean,pcm_fftMag_mfcc_sma[1]_amean,pcm_fftMag_mfcc_sma[2]_amean,pcm_fftMag_mfcc_sma[3]_amean,pcm_fftMag_mfcc_sma[4]_amean,pcm_fftMag_mfcc_sma[5]_amean,pcm_fftMag_mfcc_sma[6]_amean,pcm_fftMag_mfcc_sma[7]_amean,pcm_fftMag_mfcc_sma[8]_amean,...,CAI Trait Full Score,STAI Trait Score,Age,Gender,Lang,college,presentation,ethnicity,presentation_3_months,highest_education
0,0.054795,-0.488793,-2.748184,1.000000,-0.430783,-0.375515,-0.542790,0.680896,-1.534743,0.919486,...,0.372881,-0.172460,1.000000,0.5,1.0,0.25,0.4,0.25,0.0,1.000000
1,0.068493,-2.606460,0.051452,0.028323,0.248100,1.000000,-2.343186,0.940317,-0.426018,-0.032830,...,0.444444,0.064857,0.666667,1.0,1.0,0.25,0.2,0.25,0.5,0.666667
2,0.109589,1.000000,-1.261042,-0.160355,0.463831,-0.702390,-0.634143,0.300597,-1.298857,-0.036971,...,0.470085,0.392495,0.333333,1.0,0.5,0.25,0.6,0.50,1.0,0.333333
3,0.273973,-1.761526,0.278652,-0.223855,0.591640,-0.048735,0.181717,-0.060498,1.000000,-0.534371,...,0.159420,0.101176,0.333333,0.5,1.0,0.75,0.2,0.50,1.0,0.333333
4,0.287671,-0.242368,-0.465300,-0.183967,0.135012,-0.197349,0.376938,0.074575,-0.727869,0.712692,...,0.444444,0.289076,0.333333,0.5,0.5,0.25,0.4,0.50,0.0,0.333333
5,0.315068,-1.225563,-0.056142,0.088127,0.314762,0.696975,-1.693631,1.000000,-0.456281,-0.857318,...,0.244444,0.252941,0.333333,0.5,0.5,0.25,0.4,0.50,0.0,0.333333
6,0.438356,-0.629186,1.000000,-0.072619,1.000000,-0.542345,1.000000,0.299952,-0.249363,-0.055792,...,1.000000,-0.542017,0.666667,1.0,1.0,0.25,0.4,0.25,0.5,0.666667
7,0.479452,-1.788873,0.848149,0.132724,-0.015859,0.139432,0.973548,-0.303547,0.421996,-0.046936,...,NaN,-0.174442,0.333333,1.0,0.5,0.75,0.8,0.25,1.0,0.333333
8,0.506849,-0.274595,0.911665,0.179373,-0.092973,0.132560,0.183170,0.053444,-0.240917,-0.195748,...,0.094017,-0.271008,0.333333,0.5,NaN,1.00,0.4,0.75,0.5,0.333333
9,0.561644,-0.545830,-0.048349,0.012372,0.498597,0.478063,-3.432900,0.471068,-0.577636,-2.774909,...,0.507692,NaN,0.666667,1.0,0.5,0.25,0.6,0.75,1.0,0.666667


In [5]:
for i, col in zip(range(len(all_slopes.columns)), all_slopes.columns):
    print(i, col)

0 id
1 pcm_RMSenergy_sma_amean
2 pcm_fftMag_mfcc_sma[1]_amean
3 pcm_fftMag_mfcc_sma[2]_amean
4 pcm_fftMag_mfcc_sma[3]_amean
5 pcm_fftMag_mfcc_sma[4]_amean
6 pcm_fftMag_mfcc_sma[5]_amean
7 pcm_fftMag_mfcc_sma[6]_amean
8 pcm_fftMag_mfcc_sma[7]_amean
9 pcm_fftMag_mfcc_sma[8]_amean
10 pcm_fftMag_mfcc_sma[9]_amean
11 pcm_fftMag_mfcc_sma[10]_amean
12 pcm_fftMag_mfcc_sma[11]_amean
13 pcm_fftMag_mfcc_sma[12]_amean
14 pcm_zcr_sma_amean
15 voiceProb_sma_amean
16 F0_sma_amean
17 #pause
18 pause_frequency
19 pause_interval
20 mean
21 percent
22 jitterLocal_sma_amean
23 jitterDDP_sma_amean
24 shimmerLocal_sma_amean
25 EDA_PPT
26 HR_PPT
27 TEMP_PPT
28 BVP_PPT
29 ACC_PPT
30 IBI_PPT
31 EDA_FREQ_PPT
32 EDA_AMP_PPT
33 CAI State Score
34 CAI Trait Full Score
35 STAI Trait Score
36 Age
37 Gender
38 Lang
39 college
40 presentation
41 ethnicity
42 presentation_3_months
43 highest_education


In [44]:
#from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_predict
from scipy.stats import pearsonr

# Problem: CAI State values are showing up as the CAI Dyadic...

y_columns = ['CAI State Score', 'CAI Trait Full Score', 'STAI Trait Score']


def get_combo_predictions(X0, slope_data):
    corrs = []
    ps = []
    for y_col, y_i in zip(y_columns, range(len(y_columns))):
        y0 = slope_data[y_col].to_numpy(copy=True)
        X = np.array([X0[0]])
        y = np.array(y0[0])
        
        for i in range(1, len(X0)):  # remove NaN from data
            is_nan = False
            for x in X0[i]:
                if math.isnan(x):
                    is_nan = True
                    break
            if not math.isnan(y0[i]) and not is_nan:
                print(X0[i], y0[i])
                X = np.append(X, [X0[i]], axis=0)
                y = np.append(y, y0[i])
        
        #print(len(X))
        folds = min(10, len(X))
        model = LinearRegression()
        cv = KFold(folds, shuffle=True, random_state=42)
        predicted_vals0 = cross_val_predict(model, X, y, cv=cv)
        actual_vals0 = slope_data[y_col].to_numpy(copy=True)
        #TODO: why this for actual_vals0 isntead of the sanitized y we already got?
        predicted_vals = []
        actual_vals = []
        

        for j in range(len(predicted_vals0)):
            if not math.isnan(predicted_vals0[j]) and not math.isnan(actual_vals0[j]):
                print(predicted_vals0[j], actual_vals0[j])
                predicted_vals.append(predicted_vals0[j])
                actual_vals.append(actual_vals0[j])

        correlation, pval = pearsonr(predicted_vals, actual_vals)
        #to_print = str(correlation) + str(pval)
        #if pval < 0.15: 
        #    if not folds == 10:
        #        print("With # KFolds", folds)
        #    print("{0}:\t\t{1}\t\t{2}\t{3}\t\t{4}".format(y_col, correlation, pval, "Rows:", len(y)))
        corrs.append(correlation)
        ps.append(pval)
        
    return corrs, ps

def get_predictions(X0, slope_data):
    corrs = []
    ps = []
    for y_col, y_i in zip(y_columns, range(len(y_columns))):
        y0 = slope_data[y_col].to_numpy(copy=True)
        X = np.array(X0[0])
        y = np.array(y0[0])
        
        for i in range(1,len(X0)):  # remove NaN from data
            if math.isnan(y0[i]) or math.isnan(X0[i]):
                continue
            else:
                X = np.append(X, X0[i])
                y = np.append(y, y0[i])
            
        X = X.reshape(-1, 1)
        folds = min(10, len(X))
        model = LinearRegression()
        cv = KFold(folds, shuffle=True, random_state=42)
        predicted_vals0 = cross_val_predict(model, X, y, cv=cv)
        actual_vals0 = slope_data[y_col].to_numpy(copy=True)
        predicted_vals = []
        actual_vals = []
        

        for j in range(len(predicted_vals0)):
            if not math.isnan(predicted_vals0[j]) and not math.isnan(actual_vals0[j]):
                predicted_vals.append(predicted_vals0[j])
                actual_vals.append(actual_vals0[j])

        correlation, pval = pearsonr(predicted_vals, actual_vals)
        corrs.append(correlation)
        ps.append(pval)
        
    return corrs, ps
      
def generate_combos(slope_data):
    group_c = []
    group_p = []
    group_titles = []
    
    demo_columns = [slope_data.columns[36], slope_data.columns[38], slope_data.columns[41], slope_data.columns[43]]
    
#     group_titles.append('all demos')
#     X0_demo = slope_data[demo_columns].to_numpy(copy=True)
#     c,p = get_combo_predictions(X0_demo, slope_data)
#     group_c.append(c)
#     group_p.append(p)
    
#     group_titles.append("bios + demos")
#     columns = slope_data.columns[25:33].to_list() + demo_columns
#     X0 = slope_data[columns].to_numpy(copy=True)
#     c, p = get_combo_predictions(X0, slope_data)
#     group_c.append(c)
#     group_p.append(p)
    
#     group_titles.append('mfcc1-12 + demos')
#     columns = slope_data.columns[2:14].to_list() + demo_columns
#     X0 = slope_data[columns].to_numpy(copy=True)
#     c,p = get_combo_predictions(X0, slope_data)
#     group_c.append(c)
#     group_p.append(p)

#     group_titles.append('mfcc1-12')
#     X0_mfcc = slope_data[slope_data.columns[2:14]].to_numpy(copy=True)
#     c,p = get_combo_predictions(X0_mfcc, slope_data)
#     group_c.append(c)
#     group_p.append(p)

#     group_titles.append('pauses')
#     #X0_pauses = slope_data[slope_data.columns[17:20]].to_numpy(copy=True) # #pause, pause_frequency, pause_interval
#     columns = slope_data.columns[17:20].to_list()
#     X0_pauses = slope_data[columns].to_numpy(copy=True)
#     c, p = get_combo_predictions(X0_pauses, slope_data)
#     group_c.append(c)
#     group_p.append(p)
    
#     for col in slope_data.columns[17:20].to_list():
#         group_titles.append(col)
#         X0 = slope_data[col].to_numpy(copy=True)
#         c,p = get_predictions(X0, slope_data)
#         group_c.append(c)
#         group_p.append(p)

#     group_titles.append('mfccs + pauses')
#     columns = slope_data.columns[2:14].to_list() + slope_data.columns[17:20].to_list()
#     X0_pauses = slope_data[columns].to_numpy(copy=True) # #pause, pause_frequency, pause_interval
#     c, p = get_combo_predictions(X0_pauses, slope_data)
#     group_c.append(c)
#     group_p.append(p)

#     group_titles.append('jitter, shimmer')
#     X0_jitter = slope_data[slope_data.columns[22:25]].to_numpy(copy=True)
#     c,p = get_combo_predictions(X0_jitter, slope_data)
#     group_c.append(c)
#     group_p.append(p)
    
#     # Just the two jitter variables (they performed better)
#     group_titles.append('jitterDDP_sma_amean, jitterLocal_sma_amean')
#     columns = ['jitterDDP_sma_amean', 'jitterLocal_sma_amean']
#     X0_1 = slope_data[columns].to_numpy(copy=True)
#     c, p = get_combo_predictions(X0_1, slope_data)
#     group_c.append(c)
#     group_p.append(p)
    
#     # Jitter, shimmer individually
#     for col in slope_data.columns[22:25].to_list():
#         group_titles.append(col)
#         X0 = slope_data[col].to_numpy(copy=True)
#         c,p = get_predictions(X0, slope_data)
#         group_c.append(c)
#         group_p.append(p)
    
#     group_titles.append('jitter, shimmer, pauses')
#     columns = slope_data.columns[22:25].to_list() + slope_data.columns[17:20].to_list()
#     X0_jitter = slope_data[columns].to_numpy(copy=True)
#     c,p = get_combo_predictions(X0_jitter, slope_data)
#     group_c.append(c)
#     group_p.append(p)

#     group_titles.append('mfccs + jitter, shimmer')
#     columns = slope_data.columns[2:14].to_list() + slope_data.columns[22:25].to_list()
#     X0_jitter = slope_data[columns].to_numpy(copy=True)
#     c,p = get_combo_predictions(X0_jitter, slope_data)
#     group_c.append(c)
#     group_p.append(p)
    
    group_titles.append('mfccs + jitter shimmer + pauses')
    columns = slope_data.columns[2:14].to_list() + slope_data.columns[22:25].to_list() + slope_data.columns[17:20].to_list()
    X0_jitter = slope_data[columns].to_numpy(copy=True)
    c,p = get_combo_predictions(X0_jitter, slope_data)
    group_c.append(c)
    group_p.append(p)

    group_titles.append("all bio")
    X0_eda = slope_data[slope_data.columns[25:33]].to_numpy(copy=True)
    c,p = get_combo_predictions(X0_eda, slope_data)
    group_c.append(c)
    group_p.append(p)
    
    group_titles.append('HR_PPT, EDA_FREQ_PPT')
    columns = ['HR_PPT', 'EDA_FREQ_PPT']
    X0_0 = slope_data[columns].to_numpy(copy=True)
    c, p = get_combo_predictions(X0_0, slope_data)
    group_c.append(c)
    group_p.append(p)
    
    for col in slope_data.columns[25:33].to_list():
        group_titles.append(col)
        X0 = slope_data[col].to_numpy(copy=True)
        c,p = get_predictions(X0, slope_data)
        group_c.append(c)
        group_p.append(p)
    
    group_titles.append('bio + jitter, shimmer')
    columns = slope_data.columns[25:33].to_list() + slope_data.columns[22:25].to_list()
    X0_bio_mfcc = slope_data[columns].to_numpy(copy=True)
    c,p = get_combo_predictions(X0_bio_mfcc, slope_data)
    group_c.append(c)
    group_p.append(p)
    
    group_titles.append('bio + jitter, shimmer + pauses')
    columns = slope_data.columns[25:33].to_list() + slope_data.columns[22:25].to_list() + slope_data.columns[17:20].to_list()
    X0_bio_mfcc = slope_data[columns].to_numpy(copy=True)
    c,p = get_combo_predictions(X0_bio_mfcc, slope_data)
    group_c.append(c)
    group_p.append(p)
    
    group_titles.append('bio + pauses')
    columns = slope_data.columns[25:33].to_list() + slope_data.columns[17:20].to_list()
    X0_bio_mfcc = slope_data[columns].to_numpy(copy=True)
    c,p = get_combo_predictions(X0_bio_mfcc, slope_data)
    group_c.append(c)
    group_p.append(p)

    group_titles.append('bio + mfcc')
    columns = slope_data.columns[25:33].to_list() + slope_data.columns[2:14].to_list()
    X0_bio_mfcc = slope_data[columns].to_numpy(copy=True)
    c,p = get_combo_predictions(X0_bio_mfcc, slope_data)
    group_c.append(c)
    group_p.append(p)
    
    group_titles.append('all of em')
    columns = slope_data.columns[2:14].to_list() + slope_data.columns[25:33].to_list() + slope_data.columns[22:25].to_list() + slope_data.columns[17:20].to_list() + demo_columns
    X0 = slope_data[columns].to_numpy(copy=True)
    c, p = get_combo_predictions(X0, slope_data)
    group_c.append(c)
    group_p.append(p)
    
    return group_c, group_p, group_titles


def combos_with_demo(slope_data, demo_col):
    group_c = []
    group_p = []
    group_titles = []
    
    #demo_columns = [slope_data.columns[36], slope_data.columns[38], slope_data.columns[41], slope_data.columns[43]]

    group_titles.append('mfcc1-12')
    columns = slope_data.columns[2:14].to_list() + [demo_col]
    X0_mfcc = slope_data[columns].to_numpy(copy=True)
    c,p = get_combo_predictions(X0_mfcc, slope_data)
    group_c.append(c)
    group_p.append(p)

    group_titles.append('pauses')
    columns = slope_data.columns[17:20].to_list() + [demo_col]
    X0_pauses = slope_data[columns].to_numpy(copy=True) # #pause, pause_frequency, pause_interval
    c, p = get_combo_predictions(X0_pauses, slope_data)
    group_c.append(c)
    group_p.append(p)
    
    for col in slope_data.columns[17:20].to_list():
        group_titles.append(col)
        X0 = slope_data[col].to_numpy(copy=True)
        c,p = get_predictions(X0, slope_data)
        group_c.append(c)
        group_p.append(p)

    group_titles.append('mfccs + pauses')
    columns = slope_data.columns[2:14].to_list() + slope_data.columns[17:20].to_list() + [demo_col]
    X0_pauses = slope_data[columns].to_numpy(copy=True) # #pause, pause_frequency, pause_interval
    c, p = get_combo_predictions(X0_pauses, slope_data)
    group_c.append(c)
    group_p.append(p)

    group_titles.append('jitter, shimmer')
    columns = slope_data.columns[22:25].to_list() + [demo_col]
    X0_jitter = slope_data[columns].to_numpy(copy=True)
    c,p = get_combo_predictions(X0_jitter, slope_data)
    group_c.append(c)
    group_p.append(p)
    
    for col in slope_data.columns[22:25].to_list():
        group_titles.append(col)
        X0 = slope_data[[col, demo_col]].to_numpy(copy=True)
        c,p = get_combo_predictions(X0, slope_data)
        group_c.append(c)
        group_p.append(p)
    
    group_titles.append('jitter, shimmer, pauses')
    columns = slope_data.columns[22:25].to_list() + slope_data.columns[17:20].to_list() + [demo_col]
    X0_jitter = slope_data[columns].to_numpy(copy=True)
    c,p = get_combo_predictions(X0_jitter, slope_data)
    group_c.append(c)
    group_p.append(p)

    group_titles.append('mfccs + jitter, shimmer')
    columns = slope_data.columns[2:14].to_list() + slope_data.columns[22:25].to_list() + [demo_col]
    X0_jitter = slope_data[columns].to_numpy(copy=True)
    c,p = get_combo_predictions(X0_jitter, slope_data)
    group_c.append(c)
    group_p.append(p)
    
    group_titles.append('mfccs + jitter shimmer + pauses')
    columns = slope_data.columns[2:14].to_list() + slope_data.columns[22:25].to_list() + slope_data.columns[17:20].to_list() + [demo_col]
    X0_jitter = slope_data[columns].to_numpy(copy=True)
    c,p = get_combo_predictions(X0_jitter, slope_data)
    group_c.append(c)
    group_p.append(p)

    group_titles.append("all bio")
    columns = slope_data.columns[25:33].to_list() + [demo_col]
    X0_eda = slope_data[columns].to_numpy(copy=True)
    c,p = get_combo_predictions(X0_eda, slope_data)
    group_c.append(c)
    group_p.append(p)
    
    for col in slope_data.columns[25:33].to_list():
        group_titles.append(col)
        X0 = slope_data[[col, demo_col]].to_numpy(copy=True)
        c, p = get_combo_predictions(X0, slope_data)
        group_c.append(c)
        group_p.append(p)
    
    group_titles.append('bio + jitter, shimmer')
    columns = slope_data.columns[25:33].to_list() + slope_data.columns[22:25].to_list() + [demo_col]
    X0_eda = slope_data[columns].to_numpy(copy=True)
    c,p = get_combo_predictions(X0_eda, slope_data)
    group_c.append(c)
    group_p.append(p)
    
    group_titles.append('bio + jitter, shimmer + pauses')
    columns = slope_data.columns[25:33].to_list() + slope_data.columns[22:25].to_list() + slope_data.columns[17:20].to_list() + [demo_col]
    X0_eda = slope_data[columns].to_numpy(copy=True)
    c,p = get_combo_predictions(X0_eda, slope_data)
    group_c.append(c)
    group_p.append(p)
    
    group_titles.append('bio + pauses')
    columns = slope_data.columns[25:33].to_list() + slope_data.columns[17:20].to_list() + [demo_col]
    X0_eda = slope_data[columns].to_numpy(copy=True)
    c,p = get_combo_predictions(X0_eda, slope_data)
    group_c.append(c)
    group_p.append(p)

    group_titles.append('bio + mfcc')
    columns = slope_data.columns[2:14].to_list() + slope_data.columns[25:33].to_list() + [demo_col]
    X0_bio_mfcc = slope_data[columns].to_numpy(copy=True)
    c,p = get_combo_predictions(X0_bio_mfcc, slope_data)
    group_c.append(c)
    group_p.append(p)
    
    group_titles.append('all of em')
    columns = slope_data.columns[2:14].to_list() + slope_data.columns[25:33].to_list() + slope_data.columns[22:25].to_list() + slope_data.columns[17:20].to_list() + [demo_col]
    X0 = slope_data[columns].to_numpy(copy=True)
    c, p = get_combo_predictions(X0, slope_data)
    group_c.append(c)
    group_p.append(p)
    
    return group_c, group_p, group_titles

In [45]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

def generate_graphs(c, p, titles, specifier):
    short_y_col = ["CAI St(19)", "CAI F(18)" , "STAI T(17)"]
    correlations = np.matrix(c)
    pvalues = np.matrix(p)

    correlations = np.round(correlations, decimals=2)
    pvalues = np.round(pvalues, decimals=2)
    
    graph_title = "correlations_" + specifier + ".png"
    
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20,85))
        
    p = ax.pcolor(correlations, vmin=-0.5, vmax=0.8)
    fig.colorbar(p, ax=ax, fraction=0.05, pad=0.04)
    ax.set_xticklabels(labels=short_y_col)
    ax.set_yticklabels(labels=titles)
    plt.yticks(np.arange(0, len(titles), 1.0))
    for tick in ax.get_xticklabels():
        tick.set_rotation(30)  

    for i in range(len(short_y_col)):
        for j in range(len(titles)):
            txt = 'c=' + str(correlations[j,i]) + ', p=' + str(pvalues[j,i])
            text = ax.text(i + 0.5, j + 0.5, txt, ha="center", va="center", color="w")

    fig.tight_layout()
    plt.savefig(DATA_PATH + 'expanded_features/' + graph_title)
    plt.close()

In [34]:
for i in range(36,44):
    c, p, titles = combos_with_demo(all_slopes, all_slopes.columns[i])
    filename = "with-" + str(all_slopes.columns[i])
    generate_graphs(c, p, titles, filename)

In [5]:
#c1, p1, titles1 = combos_with_demo(all_slopes, "highest_education")
#generate_graphs(c, p, titles, "with-highestedu")
#c2, p2, titles2 = combos_with_demo(all_slopes, "highest_education")
#generate_graphs(c, p, titles, "with-highestedu")
#c3, p3, titles3 = combos_with_demo(all_slopes, "Age")
#generate_graphs(c, p, titles, "with-highestedu")

short_y_col = ["CAI St(19)", "CAI F(18)" , "STAI T(17)"]
correlations = np.matrix(c)
pvalues = np.matrix(p)

correlations = np.round(correlations, decimals=2)
pvalues = np.round(pvalues, decimals=2)
    
data = [correlations, pvalues]
graph_titles = ["correlations_" + specifier + ".png", "pvalues_" + specifier + ".png"]
    
for k in range(2):
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20,85))
        
    if k == 0:
        p = ax.pcolor(data[k], vmin=-0.5, vmax=0.8)
    else:
        p = ax.pcolor(data[k], vmin = 0.0, vmax = 1.0)
    #p = ax.matshow(data[k])
    fig.colorbar(p, ax=ax, fraction=0.05, pad=0.04)
    ax.set_xticklabels(labels=short_y_col)
    ax.set_yticklabels(labels=titles)
    plt.yticks(np.arange(0, len(titles), 1.0))
    for tick in ax.get_xticklabels():
        tick.set_rotation(30)  

    for i in range(len(short_y_col)):
        for j in range(len(titles)):
            text = ax.text(i + 0.5, j + 0.5, data[k][j, i], ha="center", va="center", color="w")

    fig.tight_layout()
    plt.savefig(DATA_PATH + 'slimmed/' + graph_titles[k])
    plt.close()

NameError: name 'c' is not defined

In [45]:
# Generate graphs showing how each variable changes over time with regards to each of the survey measures
correlations = []
pvalues = []

feature_sets = ['all demos', 'bios + demos', 'mfcc1-12 + demos', 'mfcc1-12', 'pauses', 'mfccs + pauses', 'jitter, shimmer', 'jitter, shimmer, pauses', 'mfccs + jitter, shimmer', 'mfccs + jitter shimmer + pauses', 'all bio', 'bio + jitter, shimmer', 'bio + jitter, shimmer + pauses', 'bio + pauses', 'bio + mfcc', 'everything']
demo_feature_set = ['mfcc1-12', 'pauses', 'mfccs + pauses', 'jitter, shimmer', 'jitter, shimmer, pauses', 'mfccs + jitter, shimmer', 'mfccs + jitter shimmer + pauses', 'all bio', 'bio + jitter, shimmer', 'bio + jitter, shimmer + pauses', 'bio + pauses', 'bio + mfcc', 'everything']

for i in range(2, 9):
    s = get_slopes(all_data, 1, i)
    c, p, titles = generate_combos(s)
    correlations.append(c)
    pvalues.append(p)

In [46]:
fig = plt.figure(figsize=[15,40])
fig.subplots_adjust(hspace=0.45, wspace=0.3)
for i in range(16):
    feature = [corr_set[i] for corr_set in correlations]
    cai_st = np.asarray([vals[0] for vals in feature])
    cai_f = np.asarray([vals[1] for vals in feature])
    stai_t = np.asarray([vals[2] for vals in feature])
    num_trials = np.arange(2, 9)
    
    ax = fig.add_subplot(8, 2, i+1)
    ax.set_ylim(-0.8, 0.8)
    ax.plot(num_trials, cai_st, label='CAI St')
    ax.plot(num_trials, cai_f, label='CAI Full')
    ax.plot(num_trials, stai_t, label='STAI T')
    plt.ylabel('Correlation')
    plt.xlabel('Number of trials')
    plt.title(feature_sets[i])
    ax.legend(loc="upper right")

#plt.show()
plt.savefig(DATA_PATH + 'slimmed/' + 'change-with-trials')
plt.close()

In [30]:
#c, p, titles = generate_combos(all_slopes)
#generate_graphs(c, p, titles, "general")
demo_columns = [all_slopes.columns[36], all_slopes.columns[38], all_slopes.columns[41], all_slopes.columns[43]]
#demo_columns = [36, 38, 41, 43]

for i in range(2, 9):
    #for col in demo_columns:
    graph_title = 'upto' + str(i)
    s = get_slopes(all_data, 1, i)
    c, p, titles = generate_combos(s)
    generate_graphs(c, p, titles, graph_title)

In [46]:
#all_slopes = get_slopes(all_data, 1, 8)
c, p, titles = generate_combos(all_slopes)
generate_graphs(c, p, titles, 'general2')

[ 2.22435668e-02  4.13058231e-02  3.71019573e-01  1.85630077e+00
 -2.34758469e+00  2.85707979e+00 -3.10526757e-01 -2.13223587e-02
  7.08783409e-01  5.68645538e-01 -1.08554306e-01 -6.51129994e-02
 -2.84622856e-03 -2.56411986e-03 -8.34438180e-03  1.26190476e+00
  1.86177249e-01 -1.08838373e-02] 0.3
[-0.5451716  -0.23386182  0.69363441 -1.30384777 -0.635333    0.91334053
 -0.94674265 -0.02401242  0.11506213  0.75076093 -0.53636269  0.17236991
 -0.01115132 -0.01038608 -0.01031585  3.19047619  0.45534211  0.00532506] 0.1875
[ 0.1204666  -0.32646974  0.88476478 -0.09046709  0.18205842 -0.18381747
  0.72890457 -0.34706634 -0.47071114  0.26248406 -0.32947794 -0.33663057
  0.00383956  0.00390527  0.00173657  0.10714286  0.03398526 -0.08038697] -0.03703703703703703
[-2.01157571e-01 -2.68296735e-01  2.01902877e-01 -3.66338340e-01
  3.77645620e-01  2.26589381e-01 -5.30547051e-01  4.62883425e-01
  6.12608146e-02 -4.38178250e-01 -1.04828341e-01  3.79444188e-01
  5.25845607e-04  4.67863940e-04 -9.704

[-5.80447178e-02 -1.23021713e+00 -3.95368684e-01 -8.76435099e-04
 -1.50275423e-02  9.45423276e-04 -5.43524382e-01 -7.37394811e-03
 -2.84622856e-03 -2.56411986e-03 -8.34438180e-03] 0.3
[ 2.00365617e-02 -1.41770978e+00  2.74836400e-01 -1.49872976e-02
  1.28376843e-01 -2.64459196e-02  4.51851737e-01  1.80757363e-04
 -1.11513231e-02 -1.03860770e-02 -1.03158491e-02] 0.1875
[-0.17106992  1.13661041 -0.07097062 -0.00752231  0.21429041  0.03576245
  0.53041716 -0.01560667  0.00383956  0.00390527  0.00173657] -0.03703703703703703
[-2.95163728e-01  2.68421962e-01  4.19177850e-02  1.21349658e-03
  1.33941213e-02 -1.58349737e-02 -1.15271276e+00 -3.58020666e-02
  5.25845607e-04  4.67863940e-04 -9.70433690e-05] 0.3809523809523809
[ 1.11853761e-02  5.60408356e-01 -1.41632892e-01 -1.07996745e-02
 -2.82497205e-01  1.66009295e-02  3.87993868e-01 -1.54520754e-04
 -1.91693595e-03 -1.57665205e-03 -6.31733060e-03] 0.2051282051282051
[-0.1907901  -2.72072816  0.07155864  0.005509   -0.11760581  0.02956842
 -

  1.15062126e-01  7.50760929e-01 -5.36362685e-01  1.72369908e-01] 0.1282051282051282
[-0.17106992  1.13661041 -0.07097062 -0.00752231  0.21429041  0.03576245
  0.53041716 -0.01560667  0.1204666  -0.32646974  0.88476478 -0.09046709
  0.18205842 -0.18381747  0.72890457 -0.34706634 -0.47071114  0.26248406
 -0.32947794 -0.33663057] 0.04347826086956522
[-0.29516373  0.26842196  0.04191778  0.0012135   0.01339412 -0.01583497
 -1.15271276 -0.03580207 -0.20115757 -0.26829674  0.20190288 -0.36633834
  0.37764562  0.22658938 -0.53054705  0.46288342  0.06126081 -0.43817825
 -0.10482834  0.37944419] 0.1212121212121212
[ 1.11853761e-02  5.60408356e-01 -1.41632892e-01 -1.07996745e-02
 -2.82497205e-01  1.66009295e-02  3.87993868e-01 -1.54520754e-04
 -2.42712016e-02  1.28523893e-01  4.70708715e-01  1.29379613e+00
 -1.69681054e+00  3.03842192e+00 -3.32585503e-01 -5.56815940e-01
  8.76917034e-01  1.05986199e+00 -3.21841973e-02  3.72600879e-01] 0.06666666666666667
[-0.1907901  -2.72072816  0.07155864  0.

In [37]:
all_slopes

,id,pcm_RMSenergy_sma_amean,pcm_fftMag_mfcc_sma[1]_amean,pcm_fftMag_mfcc_sma[2]_amean,pcm_fftMag_mfcc_sma[3]_amean,pcm_fftMag_mfcc_sma[4]_amean,pcm_fftMag_mfcc_sma[5]_amean,pcm_fftMag_mfcc_sma[6]_amean,pcm_fftMag_mfcc_sma[7]_amean,pcm_fftMag_mfcc_sma[8]_amean,...,CAI Trait Full Score,STAI Trait Score,Age,Gender,Lang,college,presentation,ethnicity,presentation_3_months,highest_education
0,4.0,-0.029486,-1.242208,0.999224,-0.080175,0.829578,-0.041589,-5.559595,-0.146896,0.048609,...,0.101695,-0.068182,3.0,1.0,2.0,1.0,2.0,1.0,0.0,3.0
1,5.0,-0.157234,0.023257,0.028301,0.046175,-2.209175,-0.179536,-7.677800,-0.040776,-0.001736,...,0.121212,0.025641,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0
2,8.0,0.060325,-0.570004,-0.160231,0.086326,1.551703,-0.048588,-2.454410,-0.124319,-0.001955,...,0.128205,0.155172,1.0,2.0,1.0,1.0,3.0,2.0,2.0,1.0
3,20.0,-0.106264,0.125954,-0.223682,0.110113,0.107664,0.013923,0.493971,0.095714,-0.028250,...,0.043478,0.040000,1.0,1.0,2.0,3.0,1.0,2.0,2.0,1.0
4,21.0,-0.014621,-0.210320,-0.183824,0.025128,0.435978,0.028881,-0.608911,-0.069667,0.037677,...,0.121212,0.114286,1.0,1.0,1.0,1.0,2.0,2.0,0.0,1.0
5,23.0,-0.073932,-0.025377,0.088059,0.058582,-1.539741,-0.129767,-8.165118,-0.043672,-0.045322,...,0.066667,0.100000,1.0,1.0,1.0,1.0,2.0,2.0,0.0,1.0
6,32.0,-0.037955,0.452010,-0.072563,0.186115,1.198135,0.076621,-2.449147,-0.023867,-0.002949,...,0.272727,-0.214286,2.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0
7,35.0,-0.107913,0.383372,0.132621,-0.002952,-0.308029,0.074594,2.478497,0.040391,-0.002481,...,NaN,-0.068966,1.0,2.0,1.0,3.0,4.0,1.0,2.0,1.0
8,37.0,-0.016565,0.412082,0.179234,-0.017304,-0.292849,0.014035,-0.436378,-0.023059,-0.010348,...,0.025641,-0.107143,1.0,1.0,NaN,4.0,2.0,3.0,1.0,1.0
9,41.0,-0.032927,-0.021854,0.012362,0.092796,-1.056125,-0.263031,-3.846329,-0.055288,-0.146696,...,0.138462,NaN,2.0,2.0,1.0,1.0,3.0,3.0,2.0,2.0


In [40]:
audio_data['#pause'][30:70]

30    0.4125
31    0.5875
32    0.0250
33    0.0000
34    0.0250
35    0.0125
36    0.0125
37    0.0125
38    0.0125
39    0.0375
40    0.2125
41    0.1125
42    0.1500
43    0.1625
44    0.1000
45    0.1250
46    0.2375
47    0.1125
48    0.1625
49    0.3875
50    0.0875
51    0.1750
52    0.1375
53    0.1250
54    0.3000
55    0.2250
56    0.4500
57    0.4875
58    0.3375
59    0.7000
60    0.2500
61    0.5500
62    0.3625
63    0.6750
64    0.9250
65    0.5250
66    0.7250
67    0.6875
68    0.5500
69    0.6625
Name: #pause, dtype: float64